# Ch16. 합성곱 신경망 (Convolution Neural Network)

# v01. 합성곱 신경망 (Convolution Neural Network)

- 합성곱 신경망(Convolutional Neural Network)은 이미지 처리에 탁월한 성능을 보이는 신경망이다.
- 이 책의 주제는 이미지 처리가 아니라 자연어 처리이기 때문에, 어떻게 보면 책의 범위를 벗어났다고 느낄 수도 있다.
- 하지만 합성곱 신경망으로도 자연어 처리를 할 수 있다.
- 합성곱 신경망을 이용한 자연어 처리를 배우기 전에 합성곱 신경망에 대해서 먼저 이해해보자.
- 이번 챕터에 한해서만 현재 이미지 처리를 배우고 있다고 가정하고 설명한다.

<br>

- 합성곱 신경망은 크게 **합성곱층(Convolution layer)**과 **풀링층(Pooling layer)**으로 구성된다.
- 아래의 그림은 합성곱 신경망의 일반적인 예를 보여준다.

$\quad$ ![](https://wikidocs.net/images/page/64066/convpooling.PNG)

- [http://cs231n.github.io/convolutional-networks](http://cs231n.github.io/convolutional-networks)

- 위의 그림에서 `CONV`는 합성곱 연산을 의미하고, 합성곱 연산의 결과가 활성화 함수 `ReLU`를 지난다.
- 이 두 과정을 합성곱층이라고 한다.

- 그 후에 `POOL`이라는 구간을 지나는 데 이는 풀링 연산을 의미하며 풀링층이라고 한다.

<br>

- 이번 챕터에서는 합성곱 연산과 풀링 연산의 의미에 대해서 학습한다.

<br>

## 1.1 합성곱 신경망의 대두

- 합성곱 신경망은 이미지 처리에 탁월한 성능을 보이는 신경망이다.

<br>

### 1.1.1 이미지 처리

- 이미지 처리를 하기 위해서 앞서 배운 다층 퍼셉트론을 사용할 수 있지만 한계가 있었다.
- 예를 들어, 알파벳 손글씨를 분류하는 어떤 문제가 있다고 해보자.
- 아래의 그림은 알파벳 Y를비교적 정자로 쓴 손글씨와 다소 휘갈겨 쓴 손글씨 두 개를 2차원 텐서인 행렬로 표현한 것이다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv0.png)

- 사람이 보기에는 두 그림 모두 알파벳 Y로 손쉽게 판단이 가능하다.
- 하지만 기계가 보기에는 각 픽셀마다 가진 값이 거의 상이하므로 완전히 다른 값을 가진 입력이다.

- 그런데 이미지라는 것은 위와 같이 같은 대상이라도 휘어지거나, 이동되었거나, 방향이 뒤틀렸거나 등 다양한 변형이 존재한다.
- 다층 퍼셉트론은 몇 가지 픽셀만 가밧이 달라져도 민감하게 예측에 영향을 받는다는 단점이 있다.

<br>

- 좀 더 구체적으로 살펴보자.
- 위 손글씨를 다층 퍼셉트론으로 분류한다고 하면, 이미지를 1차원 텐서인 벡터로 변환하고 다층 퍼셉트론의 입력층으로 사용해야 한다.
- 두 번째 손글씨를 다층 퍼셉트론으로 분류하기 위해서 벡터로 바꾸면 다음과 같다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv1.png)

- 1차원으로 변환된 결과는 사람이 보기에도 이게 원래 어떤 이미지였는 지 알아보기가 어렵다.
- 이는 기계도 마찬가지이다.

<br>

### 1.1.2 공간적인 구조(spatial structure) 정보

- 위와 같이 결과는 변환 전에 가지고 있던 **공간적인 구조(spatial structure) 정보**가 유실된 상태이다.
- 여기서 공간적인 구조 정보라는 것은 거리가 가까운 어떤 픽셀들끼리는 어떤 연관이 있고, 어떠 픽셀들끼리는 값이 비슷하거나 등을 포함하고 있다.
- 결국 이미지의 공간적인 구조 정보를 보존하면서 학습할 수 있는 방법이 필요해졌다.
- 이를 위해 사용하는 것이 합성곱 신경망이다.

<br>

## 1.2 채널 (Channel)

- 이미지 처리의 기본적인 용어인 채널에 대해서 간단히 정의한다.

<br>

### 1.2.1 이미지의 텐서 종류

- 기계는 글자나 이미지보다 숫자, 다시 말해 텐서를 더 잘 처리할 수 있다.
- 이미지는 `(높이, 너비, 채널)`이라는 **3차원 텐서**이다.
  - `높이` : 이미지의 세로 방향 픽셀 수
  - `너비` : 이미지의 가로 방향 픽셀 수
  - `채널` : 색 성분

- 흑백 이미지는 채널 수가 1이다.
- 각 픽셀은 0부터 255 사이의 값을 가진다.

<br>

### 1.2.2 손글씨 데이터 (흑백 이미지)

- 아래는 28 x 28 픽셀의 손글씨 데이터를 보여준다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv2.png)

- 위의 손글씨 데이터는 흑백 이미지므로 채널수가 1임을 고려하면 `(28 x 28 x 1)`의 크기를 가지는 3차원 텐서이다.

<br>

### 1.2.3 컬러 이미지

- 그렇다면 흑백이 아니라 우리가 통상적으로 접하게 되는 컬러 이미지는 어떨까?
- 컬러 이미지는 **적색(Red)**, **녹색(Green)**, 청색(Blue)** 채널 수가 3개이다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv3.png)

- 하나의 픽셀은 세 가지 색깔, 삼원색의 조합으로 이루어진다.
- 만약, 높이가 28, 너비가 28인 컬러 이미지가 있다면 이 이미지의 텐서는 `(28 x 28 x 3)`의 크기를 가지는 3차원 텐서이다.
- 채널은 때로는 **깊이(depth)**라고도 한다.
- 이 경우 이미지는 `(높이, 너비, 깊이)`라는 3차원 텐서로 표현된다고 말할 수 있을 것이다.

<br>

## 1.3 합성곱 연산 (Convolution operation)

- 합성곱층은 합성곱 연산을 통해서 **이미지의 특징을 추출**하는 역할을 한다.
- 우선, 합성곱 연산에 대해서 이해해보자.

<br>

### 1.3.1 합성곱

- 합성곱은 영어로 컨볼루션(convolution)이라고도 불린다.
- **커널(kernel)** 또는 **필터(filter)**라는 $n \times m$ 크기의 행렬로 높이$(height)$ $\times$ 너비$(width)$ 크기의 이미지를 처음부터 끝까지 겹치며 훑으면서 $n \times m$ 크기의 겹쳐지는 부분의 각 이미지와 커널의 원소의 값을 곱해서 모두 더한 값을 출력으로 하는 것을 말한다.
- 이 때, 이미지의 가장 왼쪽 위부터 가장 오른쪽까지 순차적으로 훑는다.
- 커널(kernel)은 일반적으로 3 x 3 또는 5 x 5를 사용한다.

<br>

### 1.3.2 합성곱 연산 수행 과정

- 예를 통해 이해해보자.
- 아래는 $3 \times 3$ 크기의 커널로 $5 \times 5$의 이미지 행렬에 합성곱 연산을 수행하는 과정을 보여준다.
- 한 번의 연산을 1 스탭(step)이라고 했을 때, 합성곱 연산의 네 번째 스탭까지 이미지와 식으로 표현했다.

<br>

#### 1.3.2.1 첫 번째 스탭

$\quad$ ![](https://wikidocs.net/images/page/64066/conv4.png)

- (1×1) + (2×0) + (3×1) + (2×1) + (1×0) + (0×1) + (3×0) + (0×1) + (1×0) = 6

<br>

#### 1.3.2.2 두 번째 스탭

$\quad$ ![](https://wikidocs.net/images/page/64066/conv5.png)

- (2×1) + (3×0) + (4×1) + (1×1) + (0×0) + (1×1) + (0×0) + (1×1) + (1×0) = 9

<br>

#### 1.3.2.3 세 번째 스탭

$\quad$ ![](https://wikidocs.net/images/page/64066/conv6.png)

- (3×1) + (4×0) + (5×1) + (0×1) + (1×0) + (2×1) + (1×0) + (1×1) + (0×0) = 11

<br>

#### 1.3.2.4 네 번째 스탭

$\quad$ ![](https://wikidocs.net/images/page/64066/conv7.png)

- (2×1) + (1×0) + (0×1) + (3×1) + (0×0) + (1×1) + (1×0) + (4×1) + (1×0) = 10

<br>

#### 1.3.2.5 최종 결과

- 위 연산을 총 9번의 스탭가지 마쳤다고 가정하였을 때, 최종 결과는 아래와 같다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv8.png)

<br>

### 1.3.3 특성 맵 (feature map)

- 위와 같이 입력으로부터 커널을 사용하여 합성곱 연산을 통해 나온 결과를 **특성 맵(feature map)**이라고 한다.
- 위의 예제에서는 커널의 크기가 3 x 3 이였다.
- 하지만 커널의 크기는 사용자가 정할 수 있다.

<br>

### 1.3.4 스트라이드 (stride)

- 또한 커널의 이동 범위가 위의 예제에서는 한 칸이였다.
- 이 또한 사용자가 정할 수 있다.
- 이러한 이동 범위를 **스트라이드(stride)**라고 한다.

- 아래의 예제는 스트라이드가 2일 경우에 5 x 5 이미지에 합성곱 연산을 수행하는 3 x 3 커널의 움직임을 보여준다.
- 최종적으로 2 x 2의 크기의 특성 맵을 얻는다.

$\quad$ ![](https://wikidocs.net/images/page/64066/conv9.png)

<br>

## 1.4 패딩 (Padding)

